In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
#Function to make a soup object out of a URL.

def soup_object_maker(URL):
    request = requests.get(URL).text
    soup_object = BeautifulSoup(request, 'lxml')
    return soup_object

In [ ]:
#Function to get the name of the table you are accessing. 

def get_column_names(soup_object):
    DF_Name = soup_object.find('div', class_ ='stats-header__lower__title').text
    DF_Columns_List = soup_object.thead.find_all('th')
    return DF_Name, DF_Columns_List

In [ ]:
#Function to extract the text from the soup object and add it to the dataframe.

def get_rows(soup_object):
    Rows = []
    DF_Rows = soup_object.find_all('tr')
    for columns in DF_Rows[1:]:
        clear = []
        for rows in columns.find_all('td'):
            clear.append(str(rows.text))
        Rows.append(clear)
    return Rows

In [ ]:
#Function to save the dataframe and column names to a csv.

def save_to_csv(DataFrame, DF_Name):
    File_Name = '/Users/jonathanlewis/Desktop/NCAA/{}'.format(str(DF_Name))
    DataFrame.to_csv(File_Name, mode='a', header=True)
    
#Function to save the dataframe to a csv. (rows ONLY!)

def save_to_csv_body(DataFrame, DF_Name):
    File_Name = '/Users/jonathanlewis/Desktop/NCAA/{}'.format(str(DF_Name))
    DataFrame.to_csv(File_Name, mode= 'a', header=False)

In [ ]:
# Functions to tie together all other functions.
def get_data(url_list):
    soup_object = soup_object_maker(url_list)
    DF_Name, DF_Columns_List = get_column_names(soup_object)
    DF_Rows = get_rows(soup_object)
    DF_Columns = (str(x.text)for x in DF_Columns_List)
    Data_Frame = pd.DataFrame(DF_Rows, columns=[*DF_Columns])
    save_to_csv(Data_Frame, DF_Name)

#Function the same as above, just calls the "save_to_csv_body" function instead of the "save_to_csv" function.
def get_data_body(url_list):
    soup_object = soup_object_maker(url_list)
    DF_Name, DF_Columns_List = get_column_names(soup_object)
    DF_Rows = get_rows(soup_object)
    DF_Columns = (str(x.text)for x in DF_Columns_List)
    Data_Frame = pd.DataFrame(DF_Rows, columns=[*DF_Columns])
    save_to_csv_body(Data_Frame, DF_Name)

In [ ]:
#Getting the initial list of urls to scrape.

Web_Site = 'https://www.ncaa.com/stats/football/fbs'

Stats_Dropdown_Soup_Object = soup_object_maker(Web_Site)
Stats_Dropdown_List = Stats_Dropdown_Soup_Object.find('select', class_='stats-dropdown-filter stats-select form-select')

link_list = []

for item in Stats_Dropdown_List:
    https = 'https://www.ncaa.com'
    link_list.append(https+item['value'])

In [ ]:
#Function to determine the number of pages a url has.

def page_counter(URL_List):
    soup_object = soup_object_maker(URL_List)
    LI = soup_object.find_all('li', class_='stats-pager__li')
    return(len(LI))

In [ ]:
#Because each url had "pages", this bit of code figures out how many pages there are and adds a url for each.

Pagenated_URL_List = []

for item in New_List:
    LI_Count = page_counter(item)
    LI_range = range(2, LI_Count)
    Pagenated_URL_List.append(item)
    if LI_Count > 0:
        for page in LI_range:
            Pagenated_URL_List.append(item+'/p{}'.format(page))
    else:
        pass

In [ ]:
#Calling the function on the list of urls. 

for item in Pagenated_URL_List:
    if item in link_list:
        get_data(item)
    else:
        get_data_body(item)